In [1]:
from run import *

In [18]:
valid_pd = pd.read_csv('/home/zhangkechi/workspace/riddle/dataset/valid.csv')
valid_pd.head()

,A,Q
0,逝,着手咫尺间
1,逝,折不断，扣不住。
2,逝,折不断，丢不还
3,逝,折了不扣，丢了不还
4,袂,但求温暖，不求遂心快意


In [2]:
with open('/home/zhangkechi/workspace/riddle/clone_detection/code/saved_models/1653939106.60953eval_vecs.pkl','rb') as f:
    tmp = pickle.load(f)

In [4]:
tmp['vecs'].shape

(5480, 1024)

In [6]:
tmp['ans_vecs'].shape

(5480, 1024)

In [9]:
tmp['labels'].shape

(5480,)

In [19]:
len_dataset = tmp['labels'].shape[0]

In [21]:
ans = list(valid_pd['A'])
assert len(ans) == len_dataset
for i in tqdm(range(len_dataset)):
    for j in range(i):
        i_label = tmp['labels'][i]
        j_label = tmp['labels'][j]
        i_ans = ans[i]
        j_ans = ans[j]
        if i_label == j_label:
            assert i_ans == j_ans
        else:
            assert i_ans != j_ans
ans2label = {}
for e in ans:
    if e not in ans2label:
        ans2label[e] = len(ans2label)
label2ans = {ans2label[k]:k for k in ans2label}

100%|██████████████████████████████████████████████████████████████████████████| 5480/5480 [00:07<00:00, 774.85it/s]


In [27]:
# error_pair = []
# for i in tqdm(range(len_dataset)):
#     for j in range(i):
#         i_vec = tmp['ans_vecs'][i]
#         j_vec = tmp['ans_vecs'][j]
#         i_label = tmp['labels'][i]
#         j_label = tmp['labels'][j]
#         minus_vec = sum(i_vec - j_vec)
#         if i_label == j_label:
#             assert minus_vec == 0
#         else:
#             if minus_vec == 0:
#                 error_pair.append((i_label,j_label))
# #             assert sum(i_vec - j_vec) != 0
# #             print(i_label)
# #             print(j_label)

100%|███████████████████████████████████████████████████████████████████████████| 5480/5480 [44:54<00:00,  2.03it/s]


In [32]:
# with open('/home/zhangkechi/workspace/riddle/clone_detection/code/saved_models/1653939106.60953eval_vecs.pkl_error_pair','wb') as f:
#     pickle.dump([error_pair, label2ans],f)

In [30]:
error_pair = list(set(error_pair))
for e in error_pair:
    print(e, [label2ans[ee] for ee in e])

(879, 121) ['髞', '鲔']
(1112, 1072) ['鮾', '卺']
(1204, 576) ['絾', '荅']
(1001, 267) ['膱', '祻']
(911, 178) ['餇', '鸂']
(1268, 335) ['袃', '跞']
(1072, 466) ['卺', '霫']
(673, 121) ['鸻', '鲔']
(1019, 507) ['竌', '硪']
(1249, 686) ['詧', '轳']
(184, 26) ['騛', '驦']
(1214, 283) ['捃', '肸']
(829, 743) ['阼', '觏']
(264, 33) ['鵳', '堠']
(990, 842) ['繸', '閮']
(1271, 463) ['箖', '氅']
(630, 509) ['菕', '靉']
(743, 694) ['觏', '躻']
(1204, 388) ['絾', '耵']
(1100, 245) ['鎯', '傺']
(434, 192) ['蔂', '祳']
(1063, 451) ['蘩', '胑']
(1093, 908) ['箿', '觇']
(544, 71) ['鑨', '蚖']
(1042, 186) ['簑', '睉']
(1409, 1093) ['讉', '箿']
(680, 647) ['虝', '狁']
(1397, 518) ['陴', '禐']
(463, 236) ['氅', '麅']
(1099, 388) ['魣', '耵']
(1362, 814) ['茣', '缵']
(1177, 1126) ['髠', '蟅']
(1078, 1042) ['耡', '簑']
(1154, 546) ['稑', '蕌']
(990, 134) ['繸', '莕']
(478, 442) ['莂', '鯏']
(432, 350) ['圯', '竾']
(1423, 1267) ['矀', '哜']
(429, 127) ['荙', '栝']
(1042, 894) ['簑', '秳']
(1397, 122) ['陴', '诇']
(809, 481) ['駡', '砹']
(1270, 248) ['葘', '饹']
(576, 404) ['荅', '麨']
(666,

In [33]:
scores= np.matmul(tmp['vecs'],tmp['ans_vecs'].T)

In [36]:
sort_ids=np.argsort(scores, axis=-1, kind='quicksort', order=None)[:,::-1]

In [43]:
index2ans = list(valid_pd['A'])
index2ans = {i:index2ans[i] for i in range(len(index2ans))}

In [45]:
predict_words = []
for i in tqdm(range(sort_ids.shape[0])):
    this_predict = []
    this_ids = sort_ids[i]
    for each_id in this_ids:
        if index2ans[each_id] not in this_predict:
            this_predict.append(index2ans[each_id])
    predict_words.append(this_predict)

100%|███████████████████████████████████████████████████████████████████████████| 5480/5480 [04:15<00:00, 21.41it/s]


In [50]:
mrr_index = []
ground_truth = list(valid_pd['A'])
for i in range(len(ground_truth)):
    mrr_index.append(predict_words[i].index(ground_truth[i]))

In [52]:
len([e for e in mrr_index if e < 10])

23